## Checking for Correlation
### Checking Total

This query checks for discrepancies between the `OrderTotal` in the `OrderHeaders` table and the calculated total from the `OrderDetails` table. The query identifies orders where the recorded `OrderTotal` does not match the sum of the product of `Count` and `Price` from the associated order details.


In [ ]:

SELECT oh.Id AS OrderHeaderId, oh.OrderTotal, ROUND(SUM(od.Count * od.Price), 2) AS CalculatedTotal
FROM [Onlinedata].[dbo].[OrderHeaders] oh
JOIN [Onlinedata].[dbo].[OrderDetails] od ON oh.Id = od.OrderHeaderId
GROUP BY oh.Id, oh.OrderTotal
HAVING ROUND(oh.OrderTotal, 2) <> ROUND(SUM(od.Count * od.Price), 2);


: 

| OrderHeaderId | OrderTotal | CalculatedTotal |
|---------------|------------|-----------------|
|               |            |                 |


### Make sure that if PaymentStatus is "Paid", OrderStatus must be "Completed".

In [ ]:
SELECT *
FROM [Onlinedata].[dbo].[OrderHeaders]
WHERE PaymentStatus = 'Paid' AND OrderStatus !='Completed'

: 

| Id | ApplicationUserId | OrderDate | ShippingDate | OrderTotal | OrderStatus | PaymentStatus | TrackingNumber | Carrier | PaymentDate | PaymentDueDate | SessionId | PaymentIntentId | PhoneNumber | StreetAddress | City | State | PostalCode | Name |
|----|-------------------|-----------|--------------|------------|-------------|---------------|----------------|---------|-------------|----------------|-----------|-----------------|--------------|----------------|------|-------|------------|------|
|    |                   |           |              |            |             |               |                |         |             |                |           |                 |              |                |      |       |            |      |


In [ ]:
SELECT *
FROM [Onlinedata].[dbo].[OrderHeaders]
WHERE ShippingDate < OrderDate;

: 

## Clear data
### Create a new table that excludes orders which do not meet the specified conditions

In [ ]:
CREATE TABLE [Onlinedata].[dbo].[OrderHeaders_Cleanup] (
    Id INT PRIMARY KEY,  
    ApplicationUserId NVARCHAR(450),
    OrderDate DATETIME,
    ShippingDate DATETIME,
    OrderTotal DECIMAL(18, 2),  
    OrderStatus NVARCHAR(50),
    PaymentStatus NVARCHAR(50),
    TrackingNumber NVARCHAR(50),
    Carrier NVARCHAR(50),
    PaymentDate DATETIME,
    PaymentDueDate DATETIME,
    SessionId NVARCHAR(100), 
    PaymentIntentId NVARCHAR(100),
    PhoneNumber NVARCHAR(20),
    StreetAddress NVARCHAR(100),
    City NVARCHAR(50),
    State NVARCHAR(50),
    PostalCode NVARCHAR(20),
    Name NVARCHAR(100)
);


INSERT INTO [Onlinedata].[dbo].[OrderHeaders_Cleanup]
(Id, ApplicationUserId, OrderDate, ShippingDate, OrderTotal, OrderStatus, PaymentStatus, TrackingNumber, Carrier, PaymentDate, PaymentDueDate, SessionId, PaymentIntentId, PhoneNumber, StreetAddress, City, State, PostalCode, Name)
SELECT oh.Id, oh.ApplicationUserId, oh.OrderDate, oh.ShippingDate, oh.OrderTotal, oh.OrderStatus, oh.PaymentStatus, oh.TrackingNumber, oh.Carrier, oh.PaymentDate, oh.PaymentDueDate, oh.SessionId, oh.PaymentIntentId, oh.PhoneNumber, oh.StreetAddress, oh.City, oh.State, oh.PostalCode, oh.Name
FROM [Onlinedata].[dbo].[OrderHeaders] oh
WHERE NOT (
    -- Records with payment status 'Paid' and order status 'Completed'
    (oh.OrderStatus = 'Completed' AND oh.PaymentStatus != 'Paid')
    OR
    -- Condition 2: OrderTotal is inconsistent with the total amount calculated in OrderDetails
    oh.OrderTotal != (
        SELECT ROUND(SUM(od.Count * od.Price), 2)
        FROM [Onlinedata].[dbo].[OrderDetails] od
        WHERE od.OrderHeaderId = oh.Id
        GROUP BY od.OrderHeaderId
    )
    OR
    -- Condition 3: ShippingDate is earlier than OrderDate
    oh.ShippingDate < oh.OrderDate
);



## Filter data to support promotions
### Identify high-spending users. Find users with the highest total spending. These users may receive higher coupons or special promotional emails.

In [ ]:
SELECT oh.ApplicationUserId, SUM(oh.OrderTotal) AS TotalSpent
FROM [Onlinedata].[dbo].[OrderHeaders_Cleanup] oh
GROUP BY oh.ApplicationUserId
ORDER BY TotalSpent DESC;

### Push personalized offers based on product categories. If you have product category information, you can filter out users who frequently purchase a certain type of product and push discount information for that type of product.

In [ ]:
WIP

### Filter unpaid orders. You can find orders whose order status is "Pending" but have not yet been paid, and provide specific payment reminders or reminder offers.

In [ ]:
WIP

### Identify long-term non-purchasing users. Identify users who have not placed an order for a period of time. You can push special promotions or coupons to attract them to return.

In [ ]:
WIP